In [ ]:
import os
import imageio
import numpy as np
from glob import glob
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from airxd.mask import MASK
from airxd.model import ARIXD
from airxd.dataset import Dataset, parse_imctrl

# Troubleshooting

In [ ]:
temp_path = 'other_masks/'
images = glob(temp_path + '*.tif')

i = 0

test_im = imageio.volread(images[i])
#Plot positions of all nonzero elements in test_im
nz_positions = np.nonzero(test_im)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(nz_positions[0], nz_positions[1], c='r', marker='o', s = 1)
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.title(images[i])
plt.show()

# Create/load a specific dataset

Here, we'll be utilizing the auto-masking process developed in __ to automatically generate labelled/segmented data for artifact identification.


In [ ]:
Samples = glob('data/*')

In [ ]:
Samples = glob('data/*')

for path_to_sample in Samples:

    path_to_sample = path_to_sample + '/'

    if not os.path.isdir(path_to_sample +'masks'):
        #Grabbing imctrl file and parsing its contents
        imctrl_file = glob(path_to_sample + '*.imctrl')[0]
        controls = parse_imctrl(imctrl_file)

        #Creating mask, need to specify image size here
        mask = MASK(controls=controls, shape=(2880,2880))

        #Creating mask directory
        os.mkdir(path_to_sample +'masks')
        paths = glob(path_to_sample + '*.tif')

        #Generate masked image for each sample image
        for path in paths:
            image_name = path.split('/')[-1][:-4]
            image = imageio.volread(path)
            result = mask.AutoSpotMask(image, esdmul=7.0)
            imageio.imwrite(path_to_sample + f'masks/{image_name}.tif', result)
